In [ ]:
title_list = [
    "泉質",
    "温泉",
    "温泉法",
    "温泉分析書",
    "万座温泉",
    "伊東温泉",
    "吉野温泉",
    "塩江温泉",
    "塩津温泉",
    "大子温泉",
    # '山田温泉',  # 山田温泉はユニークに決まらないのでとりあえずはずしてあります。
    # 詳細は 2.1節を読んで下さい。
    "川棚温泉",
    "指宿温泉",
    "玉造温泉",
    "登別温泉",
    "花山温泉",
    "雲仙温泉",
    "鳴子温泉",
    "鳴子温泉郷",
    "大歩危温泉",
]

In [ ]:
data_list = []

import wikipedia

wikipedia.set_lang("ja")
# wikipediaの記事の読み取り

for index, title in enumerate(title_list):
    print(index + 1, title)
    text = wikipedia.page(title, auto_suggest=False).content
    item = {"app_id": index + 1, "title": title, "text": text}
    data_list.append(item)

In [ ]:
# 資格情報の設定 (個別に設定します)

discovery_credentials = {
    "apikey": "xxxx",
    "iam_apikey_description": "xxxx",
    "iam_apikey_name": "xxxx",
    "iam_role_crn": "xxxx",
    "iam_serviceid_crn": "xxxx",
    "url": "xxxx",
}

In [ ]:
# Discovery APIの初期化

import json
import os
from ibm_watson import DiscoveryV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

version = "2019-04-30"

authenticator = IAMAuthenticator(discovery_credentials["apikey"])
discovery = DiscoveryV1(version=version, authenticator=authenticator)
discovery.set_service_url(discovery_credentials["url"])

In [ ]:
# environment_id、collection_id、configuration_id の取得
# すでにUIで1つのprivate collectionが作成済みであることが前提

# environment id の取得
environment_id = discovery.list_environments().get_result()["environments"][1][
    "environment_id"
]
print("environment_id: ", environment_id)

# collection id の取得
collection_id = discovery.list_collections(environment_id).get_result()["collections"][
    0
]["collection_id"]
print("collection_id: ", collection_id)

# configuration_idの取得
configuration_id = discovery.list_configurations(environment_id).get_result()[
    "configurations"
][0]["configuration_id"]
print("configuration_id: ", configuration_id)

In [ ]:
# 文書ロード関数
# collection_id: 対象コレクション
# sample_data: 書き込み対象テキスト (json形式の配列)
# key_name: 文書のユニークキー名称


def load_text(collection_id, sample_data, key_name):
    for item in sample_data:

        # itemごとにワークのjsonファイルを作成
        print(item)
        key = item.get(key_name)
        filename = str(key) + ".json"
        f = open(filename, "w")
        json.dump(item, f)
        f.close()

        # 書き込み可能かのチェック
        collection = discovery.get_collection(
            environment_id, collection_id
        ).get_result()
        proc_docs = collection["document_counts"]["processing"]
        while True:
            if proc_docs < 20:
                break
            print("busy. waiting..")
            time.sleep(10)
            collection = discovery.get_collection(environment_id, collection_id)
            proc_docs = collection["document_counts"]["processing"]

        # jsonファイル名を引数にDiscoveryへデータロード
        with open(filename) as f:
            add_doc = discovery.add_document(environment_id, collection_id, file=f)
        os.remove(filename)

In [ ]:
# 特定のコレクションの全文書を削除する関数
# collection_id: 対象コレクション


def delete_all_docs(collection_id):

    # 文書件数取得
    collection = discovery.get_collection(environment_id, collection_id).get_result()
    doc_count = collection["document_counts"]["available"]

    results = discovery.query(
        environment_id, collection_id, return_fields="id", count=doc_count
    ).get_result()["results"]
    ids = [item["id"] for item in results]

    for id in ids:
        print("deleting doc: id =" + id)
        discovery.delete_document(environment_id, collection_id, id)

In [ ]:
# 既存文書の全削除
delete_all_docs(collection_id)

In [ ]:
# wikipedia文書のロード
load_text(collection_id, data_list, "app_id")

In [ ]:
# リスト4.8.1

# トレーニングデータの全削除
discovery.delete_all_training_data(environment_id, collection_id)

In [ ]:
# リスト4.8.2 ー Discovery の自然言語問い合わせ（Natural Language Query）
# -----------------------------------------------------------------------------
# 本スニペットは IBM Watson Discovery（V1）に対して、日本語の自然文をそのまま
# 検索クエリとして渡す例である。キーワード検索（query=...）と異なり、
# natural_language_query は内部でクエリ解釈（意図推定・語の正規化・重要語抽出等）
# を行い、より人間の質問文に近い形で関連文書をランキングする。
#
# 【前提】
# - `discovery` は既に認証・初期化済み（4.6 節のコード参照）。
# - `environment_id`, `collection_id` は既に取得済み（4.6.2 参照）。
# - 形態素辞書や Knowledge Studio モデルを登録している場合、
#   トークン化・エンティティ正規化の結果がランキングに影響する。
#
# 【理論メモ（IR 観点）】
# - natural_language_query は BM25 / tf-idf 系統の語彙統計と
#   ML ランカー（実装詳細は非公開）を組み合わせ、語の重要度（idf）や文書長正規化、
#   フィールド構造等を加味してスコアリングするのが一般的な理解である。
# - 日本語ではトークナイザ（分かち書き）と同義語展開の設計がスコアに直結する。
#   同義語は再現率を上げる一方で idf を下げコントラストを弱める可能性がある。
#
# 【実務 Tips】
# - 返却項目は `return_fields` で最小限に絞ると I/O とメモリを節約できる。
# - 取得件数は `count=` で明示指定可能（既定は環境依存）。
# - 空結果や例外（429/5xx）は運用上リトライ/バックオフで扱うこと。
# -----------------------------------------------------------------------------

# 自然言語問い合わせ（質問文そのものを入力）
# 例: 「温泉の特徴や泉質などの分類」という意図を自然文で渡す。
query_text = "温泉の特徴や泉質などの分類"

# 検索結果で返してほしいフィールドを列挙（最小限にするのが基本）
# ここではアプリ内での識別子 app_id とタイトルのみを要求
return_fields = "app_id,title"

# Discovery への問い合わせを実行。
# - natural_language_query: 自然文（質問文）を渡す入口
# - return_fields: 出力項目の絞り込み
# - get_result(): SDK のレスポンスオブジェクトから素の dict を取得
# 補足:
#   - 取得件数を制御する場合は count=10 等を追加する。
#   - 類義語辞書やエンリッチ設定を更新した直後はインデクシング完了を待つこと。
query_results = discovery.query(
    environment_id,
    collection_id,
    natural_language_query=query_text,
    return_fields=return_fields,
).get_result()

# 実際のヒット配列は "results" キーに格納される。
# 各要素には result_metadata.score（関連度スコア）等が含まれる点も活用可能。
# 例外/空配列の取り扱いは呼び出し側で実装するのが望ましい。
res2 = query_results["results"]

# （任意のデバッグ例：必要に応じてコメント解除）
# import json
# print(json.dumps(res2, indent=2, ensure_ascii=False))

In [ ]:
# リスト4.8.3
# -----------------------------------------------------------------------------
# 問い合わせ結果（res2）を人が確認しやすい形で出力しつつ、
# Discovery の学習データ（relevance training）で用いる examples 配列を組み立てる。
#
# 【前提】
# - 直前のリスト4.8.2で実行した自然言語問い合わせの結果 dict から、
#   res2 = query_results['results'] が定義済みであること。
# - Discovery V1 の学習 API（add_training_data 等）では
#   examples の各要素に {document_id, cross_reference, relevance} を渡す。
#   - document_id: Discovery が付与した一意のドキュメント ID（item['id']）
#   - cross_reference: 任意の外部参照（ここではコーパス側で付けた app_id を使う）
#   - relevance: 人手ラベル（0/1/2 など）。ここでは 0（未ラベル）で初期化し、後で人手で上書きする想定。
#
# 【実装メモ】
# - result_metadata.score はランキング（BM25 等の語彙統計＋学習ランカー）に基づく関連度スコア。
# - result_metadata.confidence は設定やエンリッチ結果によっては付与されないことがあるため get で読む。
# - 例外や欠損に強くするため dict.get を利用している。
# -----------------------------------------------------------------------------

examples = []  # 学習用 example の入れ物

# res2: discovery.query(...).get_result()['results'] を想定
for item in res2:
    # Discovery が返す内部 ID（トレーニング API で document_id に渡す）
    document_id = item.get("id")

    # ランキングに関するメタデータ。score は基本的に常に返るが、confidence は無い場合がある
    metadata = item.get("result_metadata", {})
    score = metadata.get("score")  # 関連度スコア（大きいほど関連する）
    confidence = metadata.get("confidence")  # 存在しないケースがあるため None 許容

    # インデックス時に入れておいたメタ情報（4.6 系で投入済みの app_id/title）
    app_id = item.get("app_id")  # 学習時の cross_reference に流用
    title = item.get("title")  # デバッグ出力用

    # 学習データの 1 件分（relevance は後で人手で 1/2 などに上げる想定）
    example = {
        "document_id": document_id,  # Discovery のドキュメント ID
        "cross_reference": app_id,  # 人間に見せる参照用 ID（自前の ID）
        "relevance": 0,  # 初期ラベル（未ラベル=0）。良ければ 2、やや関連なら 1 などへ更新
    }

    # 確認用に一行で主要情報を表示（スコアで並べ替えたい場合は、事前に res2 を sort する）
    print(document_id, title, app_id, score, confidence)

    # 例の配列に追加
    examples.append(example)

# （任意）後工程のため examples を確認したい場合：
# import json; print(json.dumps(examples, indent=2, ensure_ascii=False))

In [ ]:
# リスト4.8.4

# examples配列の完成
examples[0]["relevance"] = 10
examples[1]["relevance"] = 10

for example in examples:
    print(example)

In [ ]:
# リスト 4.8.5
# -----------------------------------------------------------------------------
# ランキング学習（Discovery のトレーニングデータ登録）を実施する。
#
# 【前提（これまでの手順との対応）】
# - environment_id, collection_id …… 4.6.x 系で取得済みの有効な ID。
# - query_text …… 4.8.2 の自然言語問い合わせで用いたクエリ文字列（例：「温泉の特徴や泉質などの分類」）。
# - examples …… 4.8.3 で組み立てた配列。
#     例: [{'document_id': <Discoveryのdoc id>, 'cross_reference': <自前のapp_id等>, 'relevance': 0/1/2}, ...]
#
# 【この API 呼び出しの意味】
# - natural_language_query（自然文クエリ）に対して、関連度ラベル付きの文書例（examples）を登録する。
# - 登録された「クエリ＋例」のセットが学習データとなり、以後のランキング最適化に反映される。
#   ※ 反映は非同期（バックグラウンド学習）で行われ、反映まで時間がかかることがある。
#
# 【運用・注意点】
# - examples の document_id は Discovery が返す検索結果 item['id']（コレクション内でユニーク）を必ず使用する。
# - relevance の意味づけ（一般例）:
#     2: 強い正例（非常に関連） / 1: 正例（関連） / 0: 未ラベルまたは非関連
#   最初は 0 で登録し、人手確認後に 1/2 へ更新する運用でもよい。
# - 同一の natural_language_query に対して何度も add_training_data を実行すると重複/上書きが起こり得る。
#   必要に応じて list_training_data / delete_training_data 等で管理・クリーンアップする。
# - 429（Rate limit）や 403（権限）エラー時はレート制御や資格情報を再確認する。
# -----------------------------------------------------------------------------

train_results = discovery.add_training_data(
    environment_id,
    collection_id,
    natural_language_query=query_text,  # 学習対象の自然文クエリ
    examples=examples,  # 当該クエリに対する文書のラベル集合
).get_result()

# （任意）レスポンスの中身を確認して監査ログに残すと良い：
# - 'query_id': 学習クエリの ID
# - 'updated' : 更新時刻
# - 'examples': 受理された examples の要約
# 例：
# import json
# print(json.dumps(train_results, ensure_ascii=False, indent=2))

In [ ]:
# リスト 4.8.6

# ランキング学習結果の確認
res2 = train_results["examples"]
for item in res2:
    print(item)